In [2]:
from scipy.io import loadmat
from scipy import signal

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.cm as cm
import pylab
import math
from matplotlib.collections import LineCollection
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import ListedColormap
from scipy.fftpack import rfft, irfft, fftfreq, fft, ifft, fftshift

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from prettytable import PrettyTable

import torch

%matplotlib inline

## ChanName.mat

通道的名称

一共24个通道，rawTracePersonX之中只包含19个通道[1:8 10:16 19:20 23:24]，不包含9CM 17X3 18X2 21X1 22A2。cm（废弃通道），X1 X2 X3 A2为空通道（什么都没接）。

<img src="./source/pic1.png" width = "30%" />

In [3]:
ChanName = loadmat('./input/ChanName.mat')
chanName = np.array(ChanName['ChanName'])
chanNamex = chanName.reshape(24,)

chanName = []
for i in range(24):
    chanName.append(chanNamex[i][0][0])
chanName = np.array(chanName)
chanName

array(['EEG P3 - Pz', 'EEG C3 - Pz', 'EEG F3 - Pz', 'EEG Fz - Pz',
       'EEG F4 - Pz', 'EEG C4 - Pz', 'EEG P4 - Pz', 'EEG Cz - Pz',
       'EEG CM - Pz', 'EEG A1 - Pz', 'EEG Fp1 - Pz', 'EEG Fp2 - Pz',
       'EEG T3 - Pz', 'EEG T5 - Pz', 'EEG O1 - Pz', 'EEG O2 - Pz',
       'EEG X3 - Pz', 'EEG X2 - Pz', 'EEG F7 - Pz', 'EEG F8 - Pz',
       'EEG X1 - Pz', 'EEG A2 - Pz', 'EEG T6 - Pz', 'EEG T4 - Pz'],
      dtype='<U12')

In [51]:
# 通道：一共24个通道，此处只包含19个通道[1:8 10:16 19:20 23:24]
# 不包含9CM 17X3 18X2 21X1 22A2。cm（废弃通道）
# X1 X2 X3 A2为空通道（什么都没接）

chanNameUsed = np.append(chanName[0:8], chanName[9:16])
chanNameUsed = np.append(chanNameUsed, chanName[18:20])
chanNameUsed = np.append(chanNameUsed, chanName[22:24])

chanNameUsed[11] = 'EEG T7 - Pz'  ## T3 is now T7
chanNameUsed[12] = 'EEG P7 - Pz'  ## T5 is now P7
chanNameUsed[-2] = 'EEG P8 - Pz'  ## T6 is now P8
chanNameUsed[-1] = 'EEG T8 - Pz'  ## T4 is now T8

## raw data

具体时间在timeRawTrace.mat（1x2100）文件中。采样频率300Hz 2100个点共对应7s。每个人的timeRawTrace都是一样的。

In [52]:
timeRawTrace = loadmat('./input/timeRawTrace.mat')
timeRawTrace = np.array(timeRawTrace['timeRawTrace'])
timeRawTrace = timeRawTrace.reshape(2100,)

In [53]:
OSPerson1 = loadmat('./input/Person1/OSPerson1.mat')

os1 = OSPerson1['OS'] # OS：要分析的数据，36x52x40x54 time X freq X Trial x pair
time1 = OSPerson1['Time'] # why only 36?
freq1 = OSPerson1['fOS'] # fOS：对应的频率
track1 = OSPerson1['Track'] # true label

In [95]:
# pair54: 54个感兴趣的两个通道之间的配对
# 通道：一共24个通道，此处只包含19个通道
# [1:8 10:16 19:20 23:24]
# [1:8 9:15  16:17 18:19]
pair54 = np.array(loadmat('./input/Pair54.mat')['Pair54'])
for i in range(54):
    for j in range(2):
        if 10 <= pair54[i][j] <= 16:# & pair54[i][j] <= 16:
            pair54[i][j] -= 2
        elif 19 <= pair54[i][j] <= 20:
            pair54[i][j] -= 4
        elif 23 <= pair54[i][j] <= 24:
            pair54[i][j] -= 6
        else:
            pair54[i][j] -= 1
    #pass
pair54

array([[ 0, 10],
       [ 1, 10],
       [ 2, 10],
       [ 3, 10],
       [ 4, 10],
       [ 5, 10],
       [ 6, 10],
       [ 7, 10],
       [ 8, 10],
       [ 9, 10],
       [10, 11],
       [10, 12],
       [10, 13],
       [10, 14],
       [10, 15],
       [10, 16],
       [10, 17],
       [10, 18],
       [ 0, 18],
       [ 1, 18],
       [ 2, 18],
       [ 3, 18],
       [ 4, 18],
       [ 5, 18],
       [ 6, 18],
       [ 7, 18],
       [ 8, 18],
       [ 9, 18],
       [10, 18],
       [11, 18],
       [12, 18],
       [13, 18],
       [14, 18],
       [15, 18],
       [16, 18],
       [17, 18],
       [ 0,  6],
       [ 1,  6],
       [ 2,  6],
       [ 3,  6],
       [ 4,  6],
       [ 5,  6],
       [ 6,  7],
       [ 6,  8],
       [ 6,  9],
       [ 6, 10],
       [ 6, 11],
       [ 6, 12],
       [ 6, 13],
       [ 6, 14],
       [ 6, 15],
       [ 6, 16],
       [ 6, 17],
       [ 6, 18]], dtype=uint8)

In [76]:
# rawTracePersonX
# dataTrail: 2100x40x19 时间 x Trial x 通道，包含了各通道各 Trail 的 Rawtrace
# track: 40个trial对应的图片编号。<11的编号为记忆过的图片，>10的是没有记忆过的。顺序和 trail 对应
rawTracePerson1 = loadmat('./input/Person1/rawTracePerson1.mat')
track1 = np.array(rawTracePerson1['Track']).reshape(40,)
dataTrial1 = np.array(rawTracePerson1['dataTrial'])

rawTracePerson2 = loadmat('./input/Person2/rawTracePerson2.mat')
track2 = np.array(rawTracePerson2['Track']).reshape(10,)
dataTrial2 = np.array(rawTracePerson2['dataTrial'])

rawTracePerson3 = loadmat('./input/Person3/rawTracePerson3.mat')
track3 = np.array(rawTracePerson3['Track']).reshape(40,)
dataTrial3 = np.array(rawTracePerson3['dataTrial'])

rawTracePerson4 = loadmat('./input/Person4/rawTracePerson4.mat')
track4 = np.array(rawTracePerson4['Track']).reshape(10,)
dataTrial4 = np.array(rawTracePerson4['dataTrial'])

## Preprocessing

filter_time, filter_freq, plot

In [56]:
def filter_time(data,i):
    '''
    filter and output time domain y

    data: dataTrial1[:,n_trail,:] # (2100,19) one trail's brain wave
    i: data[:,i] # channel we want to see
    '''
    f = np.zeros((2100, 2), dtype=float)
    f[:,1] = data[:,i]
    f[:,0] = np.arange(2100)/300

    N = 2100 
    T = 7.0 / 2100.0
    x = f[:,0]
    y = f[:,1]

    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

    f_signal = rfft(y)
    W = fftfreq(y.size, d=x[1]-x[0])

    cut_f_signal = f_signal.copy()
    cut_f_signal[(W<1)] = 0 # filter all frequencies below 1

    cut_signal = irfft(cut_f_signal)
    
    return cut_signal

In [57]:
def filter_freq(data,i):
    '''
    filter and output freq domain y

    data: dataTrial1[:,n_trail,:] # (2100,19) one trail's brain wave
    i: data[:,i] # channel we want to see
    '''
    
    f = np.zeros((2100, 2), dtype=float)
    f[:,1] = data[:,i]
    f[:,0] = np.arange(2100)/300

    N = 2100 
    T = 7.0 / 2100.0 
    x = f[:,0]
    y = f[:,1]

    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

    f_signal = rfft(y)
    W = fftfreq(y.size, d=x[1]-x[0])

    cut_f_signal = f_signal.copy()
    cut_f_signal[(W<1)] = 0  # filter all frequencies below 1

    cut_signal = irfft(cut_f_signal)

    cut_signal_f = fft(cut_signal)

    return cut_signal_f

In [58]:
# plot_freq: plot_freq(track1_remember[1])
def plot_freq(n_trail): 
    '''
    randomly choose from 40 trails
    plot frequency domain of 19 channel
    '''
    size = 5
    fig = plt.figure(figsize=(size,2*size))
    data = dataTrial1[:,n_trail,:] # (2100,19)
    data = np.delete(data,7,1) # delete one row
    n_rows = 18
    n_samples = 2100

    ax = fig.add_subplot(1,1,1)
    ax.set_xlim(0, 5) # x's range: 0-7s
    ax.set_xticks(np.arange(5))
    dmin = 0
    dmax = 20
    dr = (dmax - dmin)
    y0 = dmin
    y1 = (n_rows - 1) * dr + dmax
    ax.set_ylim(y0, y1)

    N = 2100 
    T = 7.0 / 2100.0
    tf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    
    segs = [] # list of lines, each line is an array of points
    for i in range(n_rows):
        yf = 2.0/N * np.abs(filter_freq(data,i)[:N//2])
        segs.append(np.column_stack((tf, yf)))  ### filter

    offsets = np.zeros((n_rows, 2), dtype=float)
    offsets[:, 1] = np.arange(n_rows) * dr

    lines = LineCollection(segs, offsets=offsets, transOffset=None) # draw lines from segs
    ax.add_collection(lines)

    ax.set_yticks(np.arange(n_rows) * dr)
    ax.set_yticklabels(chanNameUsed)

    plt.grid(True)
    plt.show()

    
# plot_time: plot_freq(track1_not_remember[1])
def plot_time(n_trail): 
    '''
    randomly choose from 40 trails
    plot time domain of 19 channel
    '''
    
    size = 10
    fig = plt.figure(figsize=(2*size,size))
    n_rows = 18 # delete one row
    n_samples = 2100
    data = dataTrial1[:,n_trail,:] # (2100,19)
    data = np.delete(data,7,1)
    t = np.arange(2100) / 300

    ax = fig.add_subplot(1,1,1)
    ax.set_xlim(0, 7.01) # x's range: 0-7s
    ax.set_xticks(np.arange(7))
    dmin = data.min()
    dmax = data.max()
    dr = (dmax - dmin)*0.2
    y0 = dmin
    y1 = (n_rows - 1) * dr + dmax
    ax.set_ylim(y0, y1)

    segs = [] # list of lines, each line is an array of points
    for i in range(n_rows):
        segs.append(np.column_stack((t, filter_time(data,i))))  ### filter

    offsets = np.zeros((n_rows, 2), dtype=float)
    offsets[:, 1] = np.arange(n_rows) * dr

    lines = LineCollection(segs, offsets=offsets, transOffset=None) # draw lines from segs
    ax.add_collection(lines)

    ax.set_yticks(np.arange(n_rows) * dr)
    ax.set_yticklabels(chanNameUsed)

    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [59]:
label = np.zeros([40+10+40+10])
for i in range(40+10+40+10):
    if i < 40:
        if track1[i] < 11: label[i] = 0
        else: label[i] = 1
    if 39 < i < 50:
        if track2[i-40] < 11: label[i] = 0
        else: label[i] = 1
    if 49 < i < 90:
        if track3[i-50] < 11: label[i] = 0
        else: label[i] = 1
    if 89 < i < 100:
        if track4[i-90] < 11: label[i] = 0
        else: label[i] = 1

label = np.zeros([40])
for i in range(40):
    if track1[i] < 11: label[i] = 0
    else: label[i] = 1

label

array([1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [60]:
# pair
ch1 = 1 - 1
ch2 = 11 - 1
pair_time = np.zeros([40+10+40+10,2100])

for i in range(40+10+40+10):
    if i < 40:
        data = dataTrial1[:,i,:]
        pair_time[i] = filter_time(data,ch1) - filter_time(data,ch2)
    if 39 < i < 50:
        data = dataTrial2[:,i-40,:]
        pair_time[i] = filter_time(data,ch1) - filter_time(data,ch2) 
    if 49 < i < 90:
        data = dataTrial3[:,i-50,:]
        pair_time[i] = filter_time(data,ch1) - filter_time(data,ch2) 
    if 89 < i < 100:
        data = dataTrial4[:,i-90,:]
        pair_time[i] = filter_time(data,ch1) - filter_time(data,ch2)


pair_time = np.zeros([40,2100])
for i in range(40):
    data = dataTrial1[:,i,:]
    pair_time[i] = filter_time(data,ch1) - filter_time(data,ch2)


pair_time.shape

(40, 2100)

## Algorithm

In [63]:
def cut_fs(dataTrial, trial, channel):
    """ 
    eg: dataTrial1第一个人的数据。channel为处理的通道1-19,提取看到图片后的数据3-6s。滤波后重建。
    result[0]为theta， [1]为beta
    """
    data = dataTrial1[:,trial,:]  ##随便挑了一次trial,  900x19
    n_row = 19
    data_new = []
    t = np.linspace(0,3,900)
    for i in range(len(t)):
        data_new.append([t[i],data[i,channel]])
    data_new = np.array(data_new) ####### 900x2 ############
    fig1 = pylab.rcParams['figure.figsize'] = (15.0,2.0)
#     plt.plot(data_new[:,0],data_new[:,1])
#     plt.show()
    
    b,a = signal.butter(5,[8/300,16/300],'bandpass')
    theta = signal.filtfilt(b,a,data_new[:,1])
#     plt.plot(t,theta)
#     plt.title('theta')
#     plt.show()
    
    b,a = signal.butter(5,[24/300,60/300],'bandpass')
    beta = signal.filtfilt(b,a,data_new[:,1])
#     plt.plot(t,beta)
#     plt.title('beta')
#     plt.show()
    
    result = [theta,beta]
    
    N = 900
    Fs = 300
    ds = Fs/N
    yy = fft(theta)
    yf = abs(yy[:int(N)])/N
    yf = fftshift(yf) ##900
    freq = np.arange(-N/2,N/2)*ds ###900
#     plt.plot(freq, yf)
#     plt.title('theta')
#     plt.show()
    
    yy = fft(beta)
    yf = abs(yy[:int(N)])/N
    yf = fftshift(yf) ##900
    freq = np.arange(-N/2,N/2)*ds ###900
#     plt.plot(freq, yf)
#     plt.title('beta')
#     plt.show()
    
    return result

In [71]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(pair_time, label, test_size=0.2)
#print(X_test1.shape)
for i in range(40):
    data = dataTrial1[:,i,:]
#     pair_time[i] = filter_time(data,ch1) - filter_time(data,ch2)
    X_train2[i] = cut_fs(dataTrial1, i, ch1)[0] - cut_fs(dataTrial1, i, ch2)[0]


classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(2),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=100),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    DummyClassifier(strategy='most_frequent', random_state=0)]

table = PrettyTable(['classifier','score'])
for clf in classifiers:
    clf.fit(X_train1, y_train1)
    score = clf.score(X_test1, y_test1)
    table.add_row([clf,score])
#print(table)

(8, 900)


C:\Users\TSHOG\Anaconda3\envs\ML\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\TSHOG\Anaconda3\envs\ML\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\TSHOG\Anaconda3\envs\ML\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [112]:
dataTrial1 = dataTrial1[900:1800,:,:]
dataTrial1.shape  ###取3s以后的数据

(900, 40, 19)

In [65]:
result = cut_fs(dataTrial1, 7, 8)  ##for test
result[0].shape

(900,)

In [114]:
# cut_fs(dataTrial, trial, channel)
pair_time = np.zeros([40,54,900])
for i in range(40):
    for j in range(54):
        data = dataTrial1[:,i,:]
    #     pair_time[i] = filter_time(data,ch1) - filter_time(data,ch2)
        pair_time[i][j] = cut_fs(dataTrial1, i, pair54[j][0])[0] - cut_fs(dataTrial1, i, pair54[j][1])[0]


print(pair_time.shape[0])

40


In [111]:
# X_train2, X_test2, y_train2, y_test2 = train_test_split(pair_time, label, test_size=0.2)
for i in range(10):
    X_test2 = pair_time[pair_time.shape[0]/10*i:pair_time.shape[0]/10*(i+1),:,:]
    X_train2 = pair_time[,:,:]
    y_train2 = label[0:33]
    y_test2 = label[33:]
    classifiers = [
       # LogisticRegression(),
        KNeighborsClassifier(2),
        SVC(gamma=2, C=1),
        GaussianProcessClassifier(1.0 * RBF(1.0)),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        MLPClassifier(alpha=1, max_iter=100),
        AdaBoostClassifier(),
        GaussianNB(),
        QuadraticDiscriminantAnalysis(),
        DummyClassifier(strategy='most_frequent', random_state=0)]

    table = PrettyTable(['classifier','score2'])

    for clf in classifiers:
        clf.fit(X_train2, y_train2)
        #clf.fit(X_train1,y_train1)
        score2 = clf.score(X_test2, y_test2)
        #score1 = clf.score(X_test1, y_test1)
        table.add_row([clf,score2])

print(table)

ValueError: Found array with dim 3. Estimator expected <= 2.